<a href="https://colab.research.google.com/github/k-dinakaran/automation-of-wordpress-post-publication-using-AI-tools/blob/main/Develop_AI_Model_to_Automatically_Adjust_Content_for_Accessibility_Features_(e_g_%2C_Screen_Reader).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers pillow contrast-checker


ERROR: Could not find a version that satisfies the requirement contrast-checker (from versions: none)
ERROR: No matching distribution found for contrast-checker


In [ ]:
# Let's check if there's an alternative package for contrast checking or if we can implement a custom function in Python for color contrast based on WCAG standards.

from PIL import ImageColor

# Custom function to calculate color contrast based on WCAG formula
def relative_luminance(color):
    """Calculate relative luminance according to WCAG 2.0 standards"""
    rgb = ImageColor.getrgb(color)
    rgb = [channel / 255.0 for channel in rgb]

    def adjust(color_channel):
        if color_channel <= 0.03928:
            return color_channel / 12.92
        return ((color_channel + 0.055) / 1.055) ** 2.4

    return 0.2126 * adjust(rgb[0]) + 0.7152 * adjust(rgb[1]) + 0.0722 * adjust(rgb[2])

def contrast_ratio(foreground, background):
    """Calculate the contrast ratio between two colors"""
    lum1 = relative_luminance(foreground)
    lum2 = relative_luminance(background)

    lighter = max(lum1, lum2)
    darker = min(lum1, lum2)

    return (lighter + 0.05) / (darker + 0.05)

# Test the function
foreground = '#000000'  # black
background = '#d3d3d3'  # light gray

contrast = contrast_ratio(foreground, background)
contrast


14.028112748396483

In [ ]:
pip install transformers torch torchvision


In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
import requests

# Load pre-trained image captioning model and processor
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Image Accessibility: Alt Text Generation
def generate_alt_text(image_path):
    # Ensure the image is opened in RGB mode
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # Generate caption
    output_ids = model.generate(pixel_values, max_length=16, num_beams=4)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return caption

# Example usage:
image_path = 'garden.jpg'
alt_text = generate_alt_text(image_path)
print(f"Generated Alt Text: {alt_text}")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Generated Alt Text: a bouquet of flowers in the middle of a walkway 
